# This generates a file containing the MJ background estimation from the ABCD method:
## A/B = C/D 
Where,
A = Signal Region (SR)
B = Signal Region Same Sign (SR-SS)
C = Control Region (CR)
D = Control Region Same Sign (CR-SS)

$$\frac{\text{SR}}{\text{SR}_{\text{SS}}} = \frac{\text{CR}}{\text{CR}_{\text{SS}}}$$


$$ \text{SR} = \text{SR}_{\text{SS}} \text{RQCD} $$

### Signal region path$
### Multi jet control region path
### Sample used for Z-QCDjets

In [1]:
import ROOT as r
from ROOT import gStyle
import numpy as np
import ctypes
import os
from IPython.display import display 
import pandas as pd 
from histogramHelpers import *


# Define the path to the channel and the directories for the regions.
channelPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/TauhadTaulep/High-Mass/"
SR = "SR/"
SRSS = "SRSS/"
CR = "NonIsoOS/"
CRSS = "NonIsoSS/"

# Select EWjj and QCDjj samples
EWjj = "Signal_Sherpa"
QCDjj = "Ztautau_SherpaRW"

RQCDwithCRs = False

# Remove previous MJ.root file
samples=[i for i in os.listdir(channelPath+SR) if '.root' in i]
print(samples)
if "MJ.root" in samples:
    os.system("rm "+channelPath+SR+"MJ.root")

Welcome to JupyROOT 6.28/04
['MJ.root', 'Ztautau_SherpaRW.root', 'Wjets.root', 'Ztautau_MGRW.root', 'Zjets.root', 'Signal_Sherpa.root', 'Signal_PoPy.root', 'MC.root', 'VV.root', 'Higgs.root', 'ttbar.root', 'singletop.root']


In [2]:
import sys
sys.path.append('/Users/diegomac/Documents/HEP/VBF-Analysis/Scripts/')
from histogramHelpers import histogramsHighStatsZll,histogramsLowStatsZll,histogramsHighStatsZtautau,histogramsLowStatsZtautau
histos = histogramsLowStatsZtautau

In [3]:
mcSamples = [EWjj,QCDjj]
backgroundSamples = ['Wjets','VV',"ttbar",'singletop']
if "Tau" in channelPath:
    backgroundSamples += ['Higgs','Zjets']
mcSamples += backgroundSamples

# Open target file
multiJetFile =r.TFile.Open(channelPath+SR+"MJ.root", "RECREATE")

for histoName in histos:
    # Determine if histogram needs to be rebined
    rebinHistogram = False
    if len(histos[histoName])>2:
        rebinHistogram = True
    
    print(histoName,rebinHistogram)
    
    
    
    if RQCDwithCRs:
        # Calculate RQCD
        dataSubtractedHistoCR = dataSubtract(histoName,channelPath+CR,"Data",mcSamples,histos,rebin=rebinHistogram)
        dataSubtractedHistoCRSS = dataSubtract(histoName,channelPath+CRSS,"Data",mcSamples,histos,rebin=rebinHistogram)
        makeNegativeBinsZero(dataSubtractedHistoCR)
        makeNegativeBinsZero(dataSubtractedHistoCRSS)
        RQCD = dataSubtractedHistoCR.Integral(1,-1)/dataSubtractedHistoCRSS.Integral(1,-1)
    else:
        RQCD = 1.2
        UncerRQCD = 1.2*0.46
    
    # Calculate the MJ Background shape
    dataSubtractedHistoSRSS = dataSubtract(histoName,channelPath+SRSS,"Data",mcSamples,histos,rebin=rebinHistogram)
    dataSubtractedHistoSRSS.Scale(RQCD)
    makeNegativeBinsZero(dataSubtractedHistoSRSS)
    MJ = dataSubtractedHistoSRSS.Clone()
    scaleUncertainty(MJ,UncerRQCD/RQCD)
    
    # Save histogram in MJ.root file
    multiJetFile.WriteObject(MJ,histoName)
    
    
multiJetFile.Close()
    

n_bjets False
Data first bin =  48.0
Subtracting first bin=  0.09508614242076874
Subtracting first bin=  0.5476701855659485
Subtracting first bin=  0.0
Subtracting first bin=  1.7136943340301514
Subtracting first bin=  1.3097434043884277
Subtracting first bin=  0.38576793670654297
Subtracting first bin=  0.04509144276380539
Subtracting first bin=  0.47930318117141724
Data first bin after subtraction =  43.423648834228516
lepiso False
Data first bin =  2.0
Subtracting first bin=  0.0
Subtracting first bin=  0.003496086923405528
Subtracting first bin=  0.0
Subtracting first bin=  0.016555868089199066
Subtracting first bin=  0.15459147095680237
Subtracting first bin=  0.0
Subtracting first bin=  0.0
Subtracting first bin=  0.0
Data first bin after subtraction =  1.8253566026687622
n_jets_interval False
Data first bin =  48.0
Subtracting first bin=  0.09508614242076874
Subtracting first bin=  0.5476701855659485
Subtracting first bin=  0.0
Subtracting first bin=  1.7136943340301514
Subtract